In [1]:
from cognite.client.exceptions import CogniteAPIError

from tests.auth_utils import get_toolkit_client

In [2]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import NodeId, SpaceApply
from cognite.client.exceptions import CogniteException

from cognite_toolkit._cdf_tk.client.data_classes.pending_instances_ids import PendingInstanceId

In [3]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [4]:
SPACE = "sp_upgrade"

In [5]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Duplicated File Pending IDs

In [13]:
file1 = FileMetadataWrite(
    external_id="ts_case6_mismatch_ts1",
    name="Case 6 Duplicated TimesSeries Pending Ids - ts1",
)

In [14]:
file2 = FileMetadataWrite(
    external_id="ts_case6_mismatch_ts2",
    name="Case 6 Duplicated TimesSeries Pending Ids - ts2",
)

In [15]:
for file in [file1, file2]:
    try:
        client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)
    except CogniteException:
        try:
            client.data_modeling.instances.delete(NodeId(SPACE, file.external_id))
        except CogniteException as e:
            print(f"\033[91mError\033[0m {e.code} {e.message!s}")
        else:
            print("\033[92mFile node deleted successfully\033[0m")
    else:
        print(f"\033[92mFile with external_id {file.external_id} deleted successfully or did not exists\033[0m")

File with external_id ts_case6_mismatch_ts1 deleted successfully or did not exists
File with external_id ts_case6_mismatch_ts2 deleted successfully or did not exists


In [16]:
filecontent = b"This is filecontent"

In [17]:
for file in [file1, file2]:
    # Delete existing file if it exists
    client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)
    try:
        created, _ = client.files.create(file)
    except CogniteException as e:
        print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    else:
        print(f"Created file: {created.external_id}")
    client.files.upload_content_bytes(filecontent, external_id=created.external_id)

Created file: ts_case6_mismatch_ts1
Created file: ts_case6_mismatch_ts2


In [18]:
try:
    updated = client.files.set_pending_ids(
        [
            PendingInstanceId(pending_instance_id=NodeId(SPACE, file1.external_id), external_id=t.external_id)
            for t in [file1, file2]
        ]
    )
except CogniteAPIError as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    print(f"Updated {len(updated)} time series with pending IDs.")

Error 500 Internal server error. Please report this error to support@cognite.com and provide us with your request id: 030ba510-54f4-92d7-8187-664a260c172f and cluster name: az-arn-dev-002


In [12]:
try:
    update2 = client.files.set_pending_ids((SPACE, file1.external_id), external_id=file2.external_id)
except CogniteAPIError as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    print(f"Updated file {update2.external_id} with pending ID from {file1.external_id}.")

Updated file ts_case6_mismatch_ts2 with pending ID from ts_case6_mismatch_ts1.
